<a href="https://colab.research.google.com/github/aakash018/image-colorizer/blob/main/server_for_image_colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle datasets download -d aksha05/flower-image-dataset

 99% 361M/363M [00:18<00:00, 24.6MB/s]
100% 363M/363M [00:18<00:00, 20.5MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/flower-image-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
# import cv2
# import matplotlib.pyplot as plt



In [ ]:
path = '/content/image'
train_datagen = ImageDataGenerator(rescale=1. / 255)##ImageDataGenerator allows you to divide your data on the fly without consuming memory for doing so.

In [ ]:
train = train_datagen.flow_from_directory(path, target_size=(256, 256),batch_size=340,class_mode=None)

Found 733 images belonging to 1 classes.


In [ ]:
train[0]

In [ ]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

(340, 256, 256)
(340, 256, 256, 2)
(340, 256, 256, 1)
(340, 256, 256, 2)


In [ ]:
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)

In [ ]:
model = Sequential()
model.add(Conv2D(64,  (3,3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 256)       295168    
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)       590080    
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 512)       1180160   
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 512)       2

In [ ]:
model.fit(X,Y,validation_split=0.1, epochs=50, batch_size=16)

Epoch 1/50
20/20 [==============================] - 3s 157ms/step - loss: 0.0339 - accuracy: 0.6570 - val_loss: 0.0373 - val_accuracy: 0.6706
Epoch 2/50
20/20 [==============================] - 3s 155ms/step - loss: 0.0340 - accuracy: 0.6577 - val_loss: 0.0366 - val_accuracy: 0.6648
Epoch 3/50
20/20 [==============================] - 3s 156ms/step - loss: 0.0336 - accuracy: 0.6570 - val_loss: 0.0366 - val_accuracy: 0.6680
Epoch 4/50
20/20 [==============================] - 3s 156ms/step - loss: 0.0335 - accuracy: 0.6587 - val_loss: 0.0366 - val_accuracy: 0.6621
Epoch 5/50
20/20 [==============================] - 3s 156ms/step - loss: 0.0336 - accuracy: 0.6563 - val_loss: 0.0492 - val_accuracy: 0.5943
Epoch 6/50
20/20 [==============================] - 3s 159ms/step - loss: 0.0384 - accuracy: 0.6068 - val_loss: 0.0382 - val_accuracy: 0.6013
Epoch 7/50
20/20 [==============================] - 3s 157ms/step - loss: 0.0350 - accuracy: 0.6300 - val_loss: 0.0377 - val_accuracy: 0.6562
Epoch 

In [ ]:
model.save("Image_Colorization")

In [ ]:
!zip -r /content/Image_Colorization.zip /content/Image_Colorization/

  adding: content/Image_Colorization/ (stored 0%)
  adding: content/Image_Colorization/variables/ (stored 0%)
  adding: content/Image_Colorization/variables/variables.index (deflated 73%)
  adding: content/Image_Colorization/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: content/Image_Colorization/assets/ (stored 0%)
  adding: content/Image_Colorization/saved_model.pb (deflated 90%)
  adding: content/Image_Colorization/keras_metadata.pb (deflated 95%)


In [3]:
from google.colab import files
files.download('/content/Image_Colorization.zip')

FileNotFoundError: ignored

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/Image_Colorization.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.2 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=b3fac522325bc83567b961a5e6763350aec1fd5b0d0b9ff76e9ea58435de2d8d
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [6]:
model_path='/content/drive/MyDrive/Image_Colorization/content/Image_Colorization'
model = keras.models.load_model(model_path)

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from starlette.middleware import Middleware
from starlette.middleware.cors import CORSMiddleware
from pydantic import BaseModel

import os

import io, base64
from PIL import Image
from starlette.responses import FileResponse

app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ImgData(BaseModel):
    image: str

@app.get('/')
async def house():
  return "Hello World !!"

@app.post('/')
async def img(data: ImgData):
  img = Image.open(io.BytesIO(base64.decodebytes(bytes(data.image, "utf-8"))))
  img.save('my-image.jpeg')
  
  test_path = '/content/my-image.jpeg'

  color_me = []
  img = img_to_array(load_img(test_path))
  img = resize(img ,(256,256))
  color_me.append(img)
  color_me = np.array(color_me, dtype=float)

  color_me = rgb2lab(1.0/255 * color_me)[:,:,:,0]

  color_me = color_me.reshape(color_me.shape+(1,))

  output = model.predict(color_me)
  output = output * 128

  result = np.zeros((256, 256, 3))
  result[:,:,0] = color_me[0][:,:,0]
  result[:,:,1:] = output[0]
  colorOut = lab2rgb(result)
  imsave("result.png", colorOut)

  

  with open("/content/result.png", 'rb') as f:
    base64image = base64.b64encode(f.read())

  return base64image

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://113e-34-171-191-42.ngrok.io


INFO:     Started server process [110]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     122.254.88.29:0 - "OPTIONS / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 348ms/step


INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 347ms/step


/usr/local/lib/python3.8/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 8 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


...

INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 348ms/step


INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 355ms/step


INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 349ms/step


INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK
INFO:     122.254.88.29:0 - "OPTIONS / HTTP/1.1" 200 OK
1/1 [==============================] - 0s 367ms/step


INFO:     122.254.88.29:0 - "POST / HTTP/1.1" 200 OK


In [ ]:
import os
test_path = '/content/flower.jpeg'



In [ ]:
#os.listdir(test_path)##to see all the files are listed or not

In [ ]:
test = os.listdir(test_path)

In [ ]:
color_me = []
img = img_to_array(load_img(test_path))
img = resize(img ,(256,256))
color_me.append(img)
color_me = np.array(color_me, dtype=float)



In [ ]:
color_me.dtype

dtype('float64')

In [ ]:
color_me = rgb2lab(1.0/255 * color_me)[:,:,:,0]##this gives the L value of the input image
## you can write the code as 1). color_me = rgb2lab(1.0/255*color_me)
#2).                             color_me = color_me[:,:,:,0]


In [ ]:
color_me.shape ## note the size

(1, 256, 256)

In [ ]:
color_me = color_me.reshape(color_me.shape+(1,))


In [ ]:
color_me.shape ## new reshaped shape 

(1, 256, 256, 1)

In [ ]:
output = model.predict(color_me)
output = output * 128 # the output is in range of 0 to 1 so we change it into original range of ab value which is in the range of 128 

1/1 [==============================] - 0s 18ms/step


In [ ]:
# for i in range(len(output)):
result = np.zeros((256, 256, 3))
result[:,:,0] = color_me[0][:,:,0]
result[:,:,1:] = output[0]
imsave("result.png", lab2rgb(result))